In [1]:
import logging
import json
from typing import Dict
from dotenv import load_dotenv
from crewai import Agent, Crew, Task, LLM, Process
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# Load environment variables
load_dotenv()

# Set up logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)


In [20]:
from crewai import Agent, Crew, Task, LLM, Process

In [24]:
from ResumeParsingAgent import ResumeParsingAgent

In [10]:
llama_model = LLM(
    model="groq/meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [ ]:
prompt = "how to make a cup of coffee"
llama_model.call([{"role": "user", "content": prompt}])

14:09:14 - LiteLLM:DEBUG: utils.py:336 - 

2025-06-05 14:09:14,550 - DEBUG - 

14:09:14 - LiteLLM:DEBUG: utils.py:336 - Request to litellm:
2025-06-05 14:09:14,552 - DEBUG - Request to litellm:
14:09:14 - LiteLLM:DEBUG: utils.py:336 - litellm.completion(model='groq/meta-llama/llama-4-scout-17b-16e-instruct', messages=[{'role': 'user', 'content': 'how to make a cup of coffee'}], temperature=0.7, stream=False)
2025-06-05 14:09:14,552 - DEBUG - litellm.completion(model='groq/meta-llama/llama-4-scout-17b-16e-instruct', messages=[{'role': 'user', 'content': 'how to make a cup of coffee'}], temperature=0.7, stream=False)
14:09:14 - LiteLLM:DEBUG: utils.py:336 - 

2025-06-05 14:09:14,553 - DEBUG - 

14:09:14 - LiteLLM:DEBUG: litellm_logging.py:458 - self.optional_params: {}
2025-06-05 14:09:14,554 - DEBUG - self.optional_params: {}
14:09:14 - LiteLLM:DEBUG: utils.py:336 - SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
2025-06-05 14:09:14,555 - DEBUG -

"A simple yet wonderful question! Making a cup of coffee can be a straightforward process, and I'm happy to guide you through it. Here's a step-by-step guide:\n\n**Method 1: Drip Coffee Maker**\n\n1. **Get your coffee beans**: Choose your favorite coffee beans or ground coffee. If you're using whole beans, grind them to a medium-coarse grind.\n2. **Measure the coffee**: Use 1 tablespoon of coffee for every 6 ounces of water. For a standard cup, use about 10-12 grams of coffee.\n3. **Add coffee to the filter**: Place a paper filter in the drip coffee maker and add the measured coffee grounds.\n4. **Add water**: Fill the water reservoir with fresh, cold water. The recommended water temperature is between 195°F and 205°F.\n5. **Turn on the coffee maker**: Switch on the coffee maker and let it do its magic.\n6. **Wait and pour**: Wait for the coffee to brew (usually around 5-10 minutes). Once it's done, pour the coffee into your cup and enjoy!\n\n**Method 2: French Press**\n\n1. **Get your

14:09:16 - LiteLLM:DEBUG: utils.py:4245 - checking potential_model_names in litellm.model_cost: {'split_model': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'stripped_model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_stripped_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'custom_llm_provider': 'groq'}
2025-06-05 14:09:16,922 - DEBUG - checking potential_model_names in litellm.model_cost: {'split_model': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'stripped_model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_stripped_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'custom_llm_provider': 'groq'}
14:09:16 - LiteLLM:DEBUG: utils.py:4543 - model_info: {'key': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'max_tokens': 8192, 'max_input_tokens': 131072, 'max_output_tokens': 8192,

2025-06-05 14:09:16,925 - DEBUG - Logging Details LiteLLM-Success Call streaming complete
14:09:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
2025-06-05 14:09:16,926 - INFO - selected model name for cost calculation: groq/meta-llama/llama-4-scout-17b-16e-instruct
14:09:16 - LiteLLM:DEBUG: utils.py:4245 - checking potential_model_names in litellm.model_cost: {'split_model': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'stripped_model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_stripped_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'custom_llm_provider': 'groq'}
2025-06-05 14:09:16,927 - DEBUG - checking potential_model_names in litellm.model_cost: {'split_model': 'meta-llama/llama-4-scout-17b-16e-instruct', 'combined_model_name': 'groq/meta-llama/llama-4-scout-17b-16e-instruct', 'stripped_model_

In [8]:

# Resume Parsing Agent
class ResumeParsingAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="Resume Parser",
            backstory="I extract structured data from resumes with high accuracy.",
            goal="Parse resumes to extract skills, experience, education, certifications, and career gaps."
        )

    def execute_task(self, task: Task, context: list = None, tools: list = None):
        resume_content = task.description
        if not resume_content:
            raise ValueError("Resume content is required.")
        
        prompt = f"""
        You are a resume parser. Extract the following fields from the text:
        - Name
        - Education
        - Skills
        - Work experience (roles, companies, duration)

        Resume:
        {resume_content}
        """
        try:
            structured_data = self.llm.call([{"role": "user", "content": prompt}])
            
            #parsed_data = safe_parse_json(structured_data, {"error": "Failed to parse resume", "raw": structured_data})
            if "error" in structured_data:
                raise ValueError(f"Resume parsing failed: {structured_data['error']}")
            return json.dumps({ "resume": structured_data.strip() }) # Return parsed JSON directly
        except Exception as e:
            raise ValueError(f"Error parsing resume: {str(e)}")

# JD Understanding Agent
class JDUnderstandingAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="JD Parser",
            backstory="I analyze job descriptions to extract key requirements.",
            goal="Extract mandatory/optional skills, seniority, and soft skills from job descriptions."
        )

    def execute_task(self, task: Task, context: list = None, tools: list = None):
        jd_content = task.description
        if not jd_content:
            raise ValueError("JD content is required.")
        
        prompt =   f"""
        You are a JD parser. Extract:
        - Job title
        - Responsibilities
        - Required/Preferred skills

        Job Description:
        {jd_content}
        """
        try:
            structured_data = self.llm.call([{"role": "user", "content": prompt}])

            return json.dumps({ "job_description": structured_data.strip() })  # Return parsed JSON directly
        except Exception as e:
            raise ValueError(f"Error parsing JD: {str(e)}")

# Candidate-Role Matching Agent
class MatchingAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="Candidate-Role Matcher",
            backstory="I match candidates to roles based on skills and experience.",
            goal="Compute a match score between resume and job description."
        )

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
        # Debug: Log context type and content
        print(f"Debug - MatchingAgent context type: {type(context)}")
        print(f"Debug - MatchingAgent context content: {context}")
        #context = json.loads(context)
        #contexts = context.split("----------")

        #resume_data, jd_data = context.split("----------")
        #if not resume_data or not jd_data:
        #    raise ValueError(f"Empty data received. Resume: {resume_data}, JD: {jd_data}")
        
        # Ensure inputs are JSON objects

        prompt = f"""
        Compare the following resume and job_description info.
        List skills or experiences that MATCH and those that are MISSING(present in Job Description but not in Resume ).
        Respond as : {{"matched_skills": [], "missing_skills": [], "match_score": 0.0}}
        context: {context}        
        """
        try:
            match_result = self.llm.call([{"role": "user", "content": prompt}])

            return match_result  # Return parsed JSON
        except Exception as e:
            raise ValueError(f"Error matching resume and JD: {str(e)}")


class ResumeEnhancerAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="Resume Enhancer",
            backstory="I suggest improvements to the resume to make it more relevant to the JD.",
            goal="Optimize resumes based on job descriptions."
        )

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
        prompt = f"""
        Given this resume and job description, suggest improvements to the resume:
        - Highlight missing skills
        - Recommend better phrasing
        - Suggest added sections or content

        context: {context}
        """
        result = self.llm.call([{"role": "user", "content": prompt}])
        return json.dumps({"resume_enhancement": result.strip()})


class CoverLetterAgent(Agent):
    def __init__(self, llm):
        super().__init__(
            llm=llm,
            role="Cover Letter Generator",
            backstory="I write cover letters tailored to job descriptions and candidate profiles.",
            goal="Generate persuasive cover letters that address gaps and emphasize fit."
        )

    def execute_task(self, task: Task, context: dict = None, tools: list = None):
        prompt = f"""
        Generate a professional cover letter that:
        - Highlights the candidate's strengths
        - Acknowledges and bridges skill or experience gaps
        - Aligns with the job description and tone

        context: {context}
        """
        result = self.llm.call([{"role": "user", "content": prompt}])
        return json.dumps({"cover_letter": result.strip()})

        

In [5]:
resume_content = """1  Vikram Bhat  Dublin, Ireland  +353-894410686  Vikrambhat249@gmail.com   Experienced Data Scientist with 8+ years of experience specializing in large language models (LLMs), Retrieval-Augmented Generation (RAG) systems, and scalable AI infrastructure. Proven track record of building robust pipelines for RAG systems, output validation, and hallucination mitigation. Strong foundation in mathematics, statistics, and software engineering, with deep experience in deploying production-grade ML systems. Passionate about open-source innovation and advancing responsible AI practices that align with human values. Work Experience:  IBM, Dublin Data Scientist May 2019 – Present • Designed and deployed Retrieval-Augmented Generation (RAG) systems using Watsonx LLMs to enhance search accuracy and reduce query latency. • Integrated LangChain with vector stores such as Milvus and Elasticsearch to enable efficient dense retrieval in production NLP pipelines. • Developed machine learning pipelines on AWS SageMaker and IBM Watson Studio for model training, deployment, and post-deployment monitoring. • Built content indexing and retrieval infrastructure to optimize document ingestion and search efficiency. • Delivered predictive models for clients in healthcare and utilities by applying advanced feature engineering and model optimization techniques. • Automated processes to monitor, detect, and resolve data anomalies, ensuring consistent data quality and integrity. • Provided hands-on mentorship within the team. • Created interactive streamlit and R Shiny dashboards to support operational decision-making and improve stakeholder visibility. Voxpro Groups, Cork Data Analyst Jan 2018 – Apr 2019 • Streamlined data integration by extracting and transforming structured and unstructured data from diverse internal tools, improving data accuracy and accessibility. • Enhanced business insights by building predictive models, generating comprehensive reports, and creating visualization solutions using Python and Power BI. • Delivered actionable insights, enabling data-driven decision-making and improving operational efficiency. 2 Cognizant Technology Solutions, Bengaluru BI Developer Jul 2012 – Aug 2016 • Optimized ETL operations for diverse data sources, including Mainframes and SQL Servers, improving data processing efficiency and integration. • Developed and implemented robust data mappings in Hadoop environments, improving data preparation and ensuring quality for downstream analytics. • Collaborated with cross-functional teams to design scalable BI solutions, enhancing data accessibility and analysis. AI Application Development • Developed and deployed several AI applications using CrewAI, LangChain, and Llama models, including conversational chatbots, RAG pipelines, and fact-checking tools. • Authored technical blog posts on building AI applications, NLP projects, sharing insights and best practices. Visit my Medium blog for detailed write-ups: Medium Blog. Skills: • Machine Learning & AI: Supervised & Unsupervised Learning, Time Series Forecasting, NLP, Large Language Models (LLMs), Retrieval-Augmented Generation (RAG), Prompt Engineering, Model Evaluation & Safety (bias, hallucination detection) • Data Processing: Elasticsearch, Milvus, Data Warehousing, ETL Pipelines • Programming: Python, R, SQL • Visualization: R Shiny, Power BI, Streamlit, Gradio, Dash • Deep Learning Frameworks: TensorFlow, PyTorch, Hugging Face Transformers • Cloud: AWS Sagemaker, IBM Watson Studio, Docker, Git, CI/CD Pipelines Education:  MSc in Data Science and Analytics, University College Cork, Cork Graduated: October 2017 | GPA: 1:1 • Scholarship: Boole / Presidential Merit Scholarships for top international student 2016/2017 • Project: Maximized students’ experience in a university using Artificial Neural Networks and Support Vector Machines B.E. in Computer Science, Sir M.VIT, Bengaluru Graduated: June 2012 • Coursework: Relational Databases, Compiler Design, Data Structures, Advanced Mathematics and Statistics"""

In [6]:
jd_content = """Primary Responsibilities   • Analyze, review and interpret complex data through the development of case-studies, enabling the data to tell a story • Build and maintain production-ready machine-learning models • Identify drivers of patterns / behaviors uncovered in data and use these to explain business trends • Contribute to the design and realization of analytical based tools/assets to identify and monitor non-standard claim patterns • Engage with subject-matter experts to explore business problems and design solutions • Present analysis and interpretation for both operational and business review and planning • Support short and long term operational and strategic business activities through the use of data and develop recommended business solutions through research, analysis and implement when appropriate   You will be rewarded and recognized for your performance in an environment that will challenge you and give you clear direction on what it takes to succeed in your role, as well as providing development for other roles you may be interested in.  Required Qualifications   • Master's degree in Statistics, Physics, Mathematics, Engineering or similar analytic domain with a significant quantitative aspect • Proficiency in AI/ML and deep learning frameworks such us PyTorch • Proven track record of developing and deploying machine learning models in production environments
"""

In [11]:
resume_parser = ResumeParsingAgent(llm=llama_model)
jd_parser = JDUnderstandingAgent(llm=llama_model)
matcher = MatchingAgent(llm=llama_model)
resumeenhancer = ResumeEnhancerAgent(llm=llama_model)
cv=CoverLetterAgent(llm=llama_model)

# Task definitions
resume_task = Task(
    description=resume_content,
    expected_output="Structured JSON with skills, experience, education, certifications, and career gaps.",
    agent=resume_parser
)

jd_task = Task(
    description=jd_content,
    expected_output="Structured JSON with mandatory/optional skills, seniority, and soft skills.",
    agent=jd_parser
)
matching_task = Task(
        description="Match resume to JD.",
        expected_output="JSON with match score, skill matches, experience match, and gaps.",
        agent=matcher,
        context=[resume_task, jd_task]
    )

resumeenhancer_task = Task(
        description="Optimize resumes based on job descriptions.",
        expected_output="Readable text report with improvements to the resume to make it more relevant to the JD.",
        agent=resumeenhancer,
        context=[resume_task, jd_task]
    )

cv_task = Task(
        description="Generate a cover letter based on the resume and job description.",
        expected_output="Readable text cover letter tailored to the job description.",
        agent=cv,
        context=[resume_task, jd_task]
    )

'{"job_description": "Here are the extracted information:\\n\\n**Job Title:** Machine Learning Analyst\\n\\n**Responsibilities:**\\n\\n1. Analyze, review, and interpret complex data through case-studies.\\n2. Build and maintain production-ready machine-learning models.\\n3. Identify drivers of patterns/behaviors in data to explain business trends.\\n4. Contribute to designing and realizing analytical tools/assets for identifying non-standard claim patterns.\\n5. Engage with subject-matter experts to explore business problems and design solutions.\\n6. Present analysis and interpretation for operational and business review and planning.\\n7. Support short-term and long-term operational and strategic business activities through data usage.\\n\\n**Required Skills:**\\n\\n1. Master\'s degree in Statistics, Physics, Mathematics, Engineering (or similar analytic domain).\\n2. Proficiency in AI/ML frameworks like PyTorch.\\n3. Proven track record of developing and deploying machine learning m

In [ ]:
crew = Crew(
        agents=[resume_parser, jd_parser, matcher, resumeenhancer, cv],
        tasks=[resume_task, jd_task, matching_task, resumeenhancer_task, cv_task],
        name="Resume and JD Matching Crew",
        description="A crew to parse resumes, understand job descriptions, match them, and summarize the results.",
        verbose=True,
        process=Process.sequential
    )

2025-06-05 14:13:30,741 - DEBUG - Starting new HTTPS connection (1): telemetry.crewai.com:4319
2025-06-05 14:13:31,213 - DEBUG - https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2
2025-06-05 14:13:35,905 - DEBUG - https://telemetry.crewai.com:4319 "POST /v1/traces HTTP/1.1" 200 2


In [13]:
result = crew.kickoff()

14:13:27 - LiteLLM:DEBUG: utils.py:336 - 

2025-06-05 14:13:27,833 - DEBUG - 

14:13:27 - LiteLLM:DEBUG: utils.py:336 - Request to litellm:
2025-06-05 14:13:27,834 - DEBUG - Request to litellm:
14:13:27 - LiteLLM:DEBUG: utils.py:336 - litellm.completion(model='groq/meta-llama/llama-4-scout-17b-16e-instruct', messages=[{'role': 'user', 'content': '\n        You are a resume parser. Extract the following fields from the text:\n        - Name\n        - Education\n        - Skills\n        - Work experience (roles, companies, duration)\n\n        Resume:\n        1  Vikram Bhat  Dublin, Ireland  +353-894410686  Vikrambhat249@gmail.com   Experienced Data Scientist with 8+ years of experience specializing in large language models (LLMs), Retrieval-Augmented Generation (RAG) systems, and scalable AI infrastructure. Proven track record of building robust pipelines for RAG systems, output validation, and hallucination mitigation. Strong foundation in mathematics, statistics, and software engin

Debug - MatchingAgent context type: <class 'str'>
Debug - MatchingAgent context content: {"resume": "Here are the extracted fields:\n\n**Name:** Vikram Bhat\n\n**Education:**\n\n* **MSc in Data Science and Analytics**, University College Cork, Cork (Graduated: October 2017, GPA: 1:1)\n* **B.E. in Computer Science**, Sir M.VIT, Bengaluru (Graduated: June 2012)\n\n**Skills:**\n\n* Machine Learning & AI: Supervised & Unsupervised Learning, Time Series Forecasting, NLP, Large Language Models (LLMs), Retrieval-Augmented Generation (RAG), Prompt Engineering, Model Evaluation & Safety (bias, hallucination detection)\n* Data Processing: Elasticsearch, Milvus, Data Warehousing, ETL Pipelines\n* Programming: Python, R, SQL\n* Visualization: R Shiny, Power BI, Streamlit, Gradio, Dash\n* Deep Learning Frameworks: TensorFlow, PyTorch, Hugging Face Transformers\n* Cloud: AWS Sagemaker, IBM Watson Studio, Docker, Git, CI/CD Pipelines\n\n**Work Experience:**\n\n* **Data Scientist**, IBM, Dublin (May 2

2025-06-05 14:13:31,468 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Thu, 05 Jun 2025 13:13:31 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0, no-store, no-cache, must-revalidate'), (b'vary', b'Origin'), (b'x-groq-region', b'gcp-europe-west3'), (b'x-ratelimit-limit-requests', b'1000'), (b'x-ratelimit-limit-tokens', b'30000'), (b'x-ratelimit-remaining-requests', b'997'), (b'x-ratelimit-remaining-tokens', b'28307'), (b'x-ratelimit-reset-requests', b'4m18.183999999s'), (b'x-ratelimit-reset-tokens', b'3.386s'), (b'x-request-id', b'req_01jx03kagbf81rqj68pa9cz9hw'), (b'cf-cache-status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'94afed7cb87ad745-LHR'), (b'Content-Encoding', b'br'), (b'alt-svc', b'h3=":443"; ma=86400')])
2025-06-05 14:13:31,469 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP

In [123]:
print(jd_task.output.raw)

{"job_description": "Here are the extracted information:\n\n### Job Title\nMachine Learning Engineer/Mathematical Data Analyst\n\n### Responsibilities\n\u2022 Analyze, review, and interpret complex data through case-studies.\n\u2022 Build and maintain production-ready machine-learning models.\n\u2022 Identify drivers of patterns/behaviors in data to explain business trends.\n\u2022 Contribute to the design and realization of analytical tools/assets to identify non-standard claim patterns.\n\u2022 Engage with subject-matter experts to explore business problems and design solutions.\n\u2022 Present analysis and interpretation for operational and business review and planning.\n\u2022 Support short-term and long-term operational and strategic business activities through data use.\n\u2022 Develop recommended business solutions through research, analysis, and implementation.\n\n### Required/Preferred Skills\n- Master's degree in Statistics, Physics, Mathematics, Engineering or similar analyt

In [124]:
print(matching_task.output.raw)

Based on the provided resume and job description, here is the comparison of skills and experiences:

**Matched Skills:**

* Machine Learning & AI:
	+ Supervised & Unsupervised Learning
	+ Time Series Forecasting
	+ NLP
	+ Large Language Models (LLMs)
	+ Retrieval-Augmented Generation (RAG)
	+ Model Evaluation & Safety (bias, hallucination detection) - These skills match with the job description's requirements.
* Data Processing:
	+ Elasticsearch and Milvus are mentioned in both the resume and job description, indicating a possible match. However, it is not explicitly stated that these tools were used for the responsibilities listed.
* Deep Learning Frameworks:
	+ TensorFlow, PyTorch, and Hugging Face Transformers are all mentioned in the resume's skills section, which align with the job description's required skills.

**Missing Skills:**

* Statistics
* Physics
* Engineering

These fields are mentioned in the job description as preferred degrees, but not explicitly listed in the resume

In [14]:
print(result.raw)

{"cover_letter": "Here is a professional cover letter that highlights the candidate's strengths, acknowledges and bridges skill or experience gaps, and aligns with the job description and tone:\n\nDear Hiring Manager,\n\nI am excited to apply for the Data Scientist - Machine Learning Engineer position at [Company Name]. With a strong educational background in Data Science and Analytics, and extensive experience in developing and deploying machine learning models, I am confident that I can make a valuable contribution to your team.\n\nAs a highly motivated and detail-oriented data scientist with a Master's degree in Data Science and Analytics from University College Cork, I possess a solid foundation in machine learning, statistics, and programming. My academic background, combined with my industry experience, has equipped me with a unique blend of technical and analytical skills. I am well-versed in AI/ML and deep learning frameworks, including PyTorch, TensorFlow, and Hugging Face Tra